In [1]:
import numpy as np
import networkx as nx
import itertools

import src.utils as ut
from src.legacy.SCMMappings_1_0 import Abstraction

from pgmpy.models import BayesianNetwork as BN
from pgmpy.factors.discrete import TabularCPD as cpd
from pgmpy.inference import VariableElimination

In [2]:
np.random.seed(0)
verbose = False

## Base model M0

In [3]:
M0 = BN([('Env','Smoke'),('Env','Cancer'),('Smoke','Cancer')])

cpdE = cpd(variable='Env',
          variable_card=2,
          values=[[.8],[.2]],
          evidence=None,
          evidence_card=None)
cpdS = cpd(variable='Smoke',
          variable_card=2,
          values=[[.8,.6],[.2,.4]],
          evidence=['Env'],
          evidence_card=[2])
cpdC = cpd(variable='Cancer',
          variable_card=2,
          values=[[.9,.8,.4,.3],[.1,.2,.6,.7]],
          evidence=['Smoke','Env'],
          evidence_card=[2,2])

M0.add_cpds(cpdE,cpdS,cpdC)
M0.check_model()

True

### Distributions over M0

In [4]:
inferM0 = VariableElimination(M0)
M0_P_ESC = inferM0.query(['Env','Smoke','Cancer'], show_progress=False)
M0_P_SC = inferM0.query(['Smoke','Cancer'], show_progress=False)
M0_P_ES = inferM0.query(['Env','Smoke'], show_progress=False)
M0_P_EC = inferM0.query(['Env','Cancer'], show_progress=False)
M0_P_S = inferM0.query(['Smoke'], show_progress=False)
M0_P_C = inferM0.query(['Cancer'], show_progress=False)
M0_P_E = inferM0.query(['Env'], show_progress=False)
M0_P_C_givenS = M0_P_SC/M0_P_S
M0_P_C_givenE = M0_P_EC/M0_P_E
M0_P_S_givenC = M0_P_SC/M0_P_C
M0_P_C_givenES = M0_P_ESC/M0_P_ES

M0do = M0.do(['Smoke'])
infer0do = VariableElimination(M0do)
M0_P_C_doS0 = infer0do.query(['Cancer'], evidence={'Smoke':0}, show_progress=False)
M0_P_C_doS1 = infer0do.query(['Cancer'], evidence={'Smoke':1}, show_progress=False)

In [5]:
print(M0_P_S_givenC)

+----------+-----------+---------------------+
| Smoke    | Cancer    |   phi(Smoke,Cancer) |
+==========+===========+=====================+
| Smoke(0) | Cancer(0) |              0.8842 |
+----------+-----------+---------------------+
| Smoke(0) | Cancer(1) |              0.3667 |
+----------+-----------+---------------------+
| Smoke(1) | Cancer(0) |              0.1158 |
+----------+-----------+---------------------+
| Smoke(1) | Cancer(1) |              0.6333 |
+----------+-----------+---------------------+


## Abstracted Model M1

In [6]:
M1 = BN([('Smoke_','Cancer_')])

cpdS_ = cpd(variable='Smoke_',
          variable_card=2,
          values=[[.2],[.8]],
          evidence=None,
          evidence_card=None)
cpdC_ = cpd(variable='Cancer_',
          variable_card=2,
          values=[[.88,.38],[.12,.62]],
          evidence=['Smoke_'],
          evidence_card=[2])

M1.add_cpds(cpdS_,cpdC_)
M1.check_model()

True

### Distributions over M1 

In [7]:
inferM1 = VariableElimination(M1)
M1_P_SC = inferM1.query(['Smoke_','Cancer_'], show_progress=False)
M1_P_S = inferM1.query(['Smoke_'], show_progress=False)
M1_P_C = inferM1.query(['Cancer_'], show_progress=False)
M1_P_C_givenS = M1_P_SC/M1_P_S
M1_P_S_givenC = M1_P_SC/M1_P_C

M1do = M1.do(['Smoke_'])
infer1do = VariableElimination(M1do)
M1_P_C_doS0 = infer1do.query(['Cancer_'], evidence={'Smoke_':0}, show_progress=False)
M1_P_C_doS1 = infer1do.query(['Cancer_'], evidence={'Smoke_':1}, show_progress=False)

In [8]:
print(M1_P_S_givenC)

+-----------+------------+-----------------------+
| Smoke_    | Cancer_    |   phi(Smoke_,Cancer_) |
+===========+============+=======================+
| Smoke_(0) | Cancer_(0) |                0.3667 |
+-----------+------------+-----------------------+
| Smoke_(0) | Cancer_(1) |                0.0462 |
+-----------+------------+-----------------------+
| Smoke_(1) | Cancer_(0) |                0.6333 |
+-----------+------------+-----------------------+
| Smoke_(1) | Cancer_(1) |                0.9538 |
+-----------+------------+-----------------------+


### Abstraction $\alpha$: M0 ->M1

In [9]:
R = ['Smoke', 'Cancer']
a = {'Smoke': 'Smoke_',
    'Cancer': 'Cancer_'}
alphas = {'Smoke_': np.eye(2),
          'Cancer_': np.eye(2)}

In [10]:
A = Abstraction(M0,M1,R,a,alphas)

In [11]:
A.evaluate_abstraction_error()

[[1. 0.]
 [0. 1.]]


[3.650024149988857e-09]

In [12]:
A.evaluate_info_loss()

['Smoke_', 'Cancer_']


array([0.44320847])

### Inverse $\alpha^{*}$

In [13]:
_,_,invalpha = A.compute_joints_and_invalpha()
print(invalpha)

['Smoke_', 'Cancer_']
[[0.5 0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0.5]
 [0.5 0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0.5]]


### Abstraction $\beta$: M0 ->M1

In [14]:
R = ['Smoke', 'Cancer']
a = {'Smoke': 'Smoke_',
    'Cancer': 'Cancer_'}
alphas = {'Smoke_': np.array([[0,1.],[1.,0]]),
          'Cancer_': np.array([[0,1.],[1.,0]])}

In [15]:
A = Abstraction(M0,M1,R,a,alphas)

In [16]:
A.evaluate_abstraction_error()

[[0. 1.]
 [1. 0.]]


[0.2164268599328641]

In [17]:
A.evaluate_info_loss()

['Smoke_', 'Cancer_']


array([0.31430371])

### Inverse $\beta^{*}$

In [18]:
_,_,invbeta = A.compute_joints_and_invalpha()
print(invbeta)

['Smoke_', 'Cancer_']
[[0.  0.  0.  0.5]
 [0.  0.  0.5 0. ]
 [0.  0.5 0.  0. ]
 [0.5 0.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.  0.  0.5 0. ]
 [0.  0.5 0.  0. ]
 [0.5 0.  0.  0. ]]


## Abstracted Model M2

In [19]:
M2 = BN([('Smoke_','Cancer_')])

cpdS_ = cpd(variable='Smoke_',
          variable_card=2,
          values=[[.2],[.8]],
          evidence=None,
          evidence_card=None)
cpdC_ = cpd(variable='Cancer_',
          variable_card=2,
          values=[[.8,.3],[.2,.7]],
          evidence=['Smoke_'],
          evidence_card=[2])

M2.add_cpds(cpdS_,cpdC_)
M2.check_model()

True

### Abstraction $\alpha$: M0 ->M2

In [20]:
R = ['Smoke', 'Cancer']
a = {'Smoke': 'Smoke_',
    'Cancer': 'Cancer_'}
alphas = {'Smoke_': np.eye(2),
          'Cancer_': np.eye(2)}

In [21]:
A = Abstraction(M0,M2,R,a,alphas)

In [22]:
A.evaluate_abstraction_error()

[[1. 0.]
 [0. 1.]]


[0.07749949674607028]

In [23]:
A.evaluate_info_loss()

['Smoke_', 'Cancer_']


array([0.44531366])

## Abstracted Model M3

In [24]:
M3 = BN([('Smoke_','Cancer_')])

cpdS_ = cpd(variable='Smoke_',
          variable_card=2,
          values=[[.8],[.2]],
          evidence=None,
          evidence_card=None)
cpdC_ = cpd(variable='Cancer_',
          variable_card=2,
          values=[[.88,.38],[.12,.62]],
          evidence=['Smoke_'],
          evidence_card=[2])

M3.add_cpds(cpdS_,cpdC_)
M3.check_model()

True

### Abstraction $\alpha$: M0 ->M3

In [25]:
R = ['Smoke', 'Cancer']
a = {'Smoke': 'Smoke_',
    'Cancer': 'Cancer_'}
alphas = {'Smoke_': np.eye(2),
          'Cancer_': np.eye(2)}

In [26]:
A = Abstraction(M0,M3,R,a,alphas)

In [27]:
A.evaluate_abstraction_error()

[[1. 0.]
 [0. 1.]]


[3.650024149988857e-09]

In [28]:
A.evaluate_info_loss()

['Smoke_', 'Cancer_']


array([0.24399835])

## Abstracted singleton model Ms

In [29]:
Ms = BN()
Ms.add_node('Star')

cpdS = cpd(variable='Star',
          variable_card=1,
          values=[[1.]],
          evidence=None,
          evidence_card=None)

Ms.add_cpds(cpdS)
Ms.check_model()

True

### Abstraction $\gamma$: M0 -> Ms 

In [30]:
R = ['Env','Smoke', 'Cancer']
a = {'Env': 'Star',
    'Smoke': 'Star',
    'Cancer': 'Star'}
alphas = {'Star': np.ones(shape=(1,8))}

In [31]:
As = Abstraction(M0,Ms,R,a,alphas)

In [32]:
As.evaluate_abstraction_error()

[]

In [33]:
As.evaluate_info_loss(verbose=True)

Alpha^-1: [[0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]]
['Star']
M0 joint: [[0.576]
 [0.064]
 [0.064]
 [0.096]
 [0.096]
 [0.024]
 [0.024]
 [0.056]]
M1 joint: [[1.]]
Transformed M1 joint: [[0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]]


array([0.36715392])

### Inverse $\gamma^{*}$

In [34]:
_,_,invgamma = As.compute_joints_and_invalpha()
print(invgamma)

['Star']
[[0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]
 [0.125]]


## Evaluating all abstractions from M0 to M1

### Support functions

In [35]:
def enumerate_all_surjective_maps(dom,codom):
    diff = dom - codom
    surjective = np.arange(0,codom,dtype=int)
    nonsurjective = np.array(list(itertools.combinations(surjective,diff)))
    
    surjectives = np.tile(surjective,(nonsurjective.shape[0],1))
    if nonsurjective.size:
        mappings = np.hstack((surjectives,nonsurjective))
    else:
        mappings = surjectives
    
    allmaps = []
    for m in mappings:
        ms = list(itertools.permutations(m))
        allmaps.extend(ms)
    
    return set(allmaps)

In [36]:
def transform_list_into_matrix(l,codom):
    alpha = np.zeros((codom,len(l)))
    for i in range(len(l)):
        alpha[l[i],i] = 1
    return alpha

In [37]:
def produce_all_abstraction_matrices(dom,codom):
    ls = enumerate_all_surjective_maps(dom,codom)
    
    Ms = []
    for l in ls:
        Ms.append(transform_list_into_matrix(l,codom))
    return Ms

### Enumeration and evalution of abstractions

In [38]:
R = ['Smoke', 'Cancer']
a = {'Smoke': 'Smoke_',
    'Cancer': 'Cancer_'}

In [39]:
alphas_S_ = produce_all_abstraction_matrices(2,2)
alphas_C_ = produce_all_abstraction_matrices(2,2)

In [40]:
for alpha_S_ in alphas_S_:
    for alpha_C_ in alphas_C_:
        alphas = {'Smoke_': alpha_S_,
                 'Cancer_': alpha_C_}

        A = Abstraction(M0,M1,R,a,alphas)
        
        e = A.evaluate_abstraction_error()
        i = A.evaluate_info_loss()
        
        print('\nalpha_S_:\n{0}, \nalpha_C_:\n{1}'.format(alpha_S_,alpha_C_))
        print('e(alpha) = {0}'.format(e))
        print('i(alpha) = {0}'.format(i))
        
        joint_M0,joint_M1,invalpha = A.compute_joints_and_invalpha()
        #print(joint_M0)
        #print(np.dot(invalpha,joint_M1))
        #print(np.sum(np.abs(joint_M0 - np.dot(invalpha,joint_M1))))

[[1. 0.]
 [0. 1.]]
['Smoke_', 'Cancer_']

alpha_S_:
[[1. 0.]
 [0. 1.]], 
alpha_C_:
[[1. 0.]
 [0. 1.]]
e(alpha) = [3.650024149988857e-09]
i(alpha) = [0.44320847]
['Smoke_', 'Cancer_']
[[1. 0.]
 [0. 1.]]
['Smoke_', 'Cancer_']

alpha_S_:
[[1. 0.]
 [0. 1.]], 
alpha_C_:
[[0. 1.]
 [1. 0.]]
e(alpha) = [0.5711586375843717]
i(alpha) = [0.54873453]
['Smoke_', 'Cancer_']
[[0. 1.]
 [1. 0.]]
['Smoke_', 'Cancer_']

alpha_S_:
[[0. 1.]
 [1. 0.]], 
alpha_C_:
[[1. 0.]
 [0. 1.]]
e(alpha) = [0.3787626192810662]
i(alpha) = [0.43432526]
['Smoke_', 'Cancer_']
[[0. 1.]
 [1. 0.]]
['Smoke_', 'Cancer_']

alpha_S_:
[[0. 1.]
 [1. 0.]], 
alpha_C_:
[[0. 1.]
 [1. 0.]]
e(alpha) = [0.2164268599328641]
i(alpha) = [0.31430371]
['Smoke_', 'Cancer_']


### Analysis of the distributions

In [41]:
alphas1 = {'Smoke_': alphas_S_[0],
         'Cancer_': alphas_C_[0]}
A1 = Abstraction(M0,M1,R,a,alphas1)
joint_M0,joint_M1_1,invalpha_1 = A1.compute_joints_and_invalpha()

alphas4 = {'Smoke_': alphas_S_[1],
         'Cancer_': alphas_C_[1]}
A4 = Abstraction(M0,M1,R,a,alphas4)
joint_M0,joint_M1_4,invalpha_4 = A4.compute_joints_and_invalpha()

['Smoke_', 'Cancer_']
['Smoke_', 'Cancer_']


In [42]:
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD

p0 = JPD(['E','S','C'],[2,2,2],joint_M0)
p1_1 = JPD(['E','S','C'],[2,2,2],np.dot(invalpha_1,joint_M1_1))
p1_4 = JPD(['E','S','C'],[2,2,2],np.dot(invalpha_4,joint_M1_4))

In [43]:
print(p1_1)

+------+------+------+------------+
| E    | S    | C    |   P(E,S,C) |
+======+======+======+============+
| E(0) | S(0) | C(0) |     0.0880 |
+------+------+------+------------+
| E(0) | S(0) | C(1) |     0.0120 |
+------+------+------+------------+
| E(0) | S(1) | C(0) |     0.1520 |
+------+------+------+------------+
| E(0) | S(1) | C(1) |     0.2480 |
+------+------+------+------------+
| E(1) | S(0) | C(0) |     0.0880 |
+------+------+------+------------+
| E(1) | S(0) | C(1) |     0.0120 |
+------+------+------+------------+
| E(1) | S(1) | C(0) |     0.1520 |
+------+------+------+------------+
| E(1) | S(1) | C(1) |     0.2480 |
+------+------+------+------------+


In [44]:
marg = ['S','C']
cond = [('C',0)]


p1 = JPD(['E','S','C'],[2,2,2],joint_M0)
p1.marginal_distribution(marg)
p1.conditional_distribution(cond)

p1_1 = JPD(['E','S','C'],[2,2,2],np.dot(invalpha_1,joint_M1_1))
p1_1.marginal_distribution(marg)
p1_1.conditional_distribution(cond)
p1_4 = JPD(['E','S','C'],[2,2,2],np.dot(invalpha_4,joint_M1_4))
p1_4.marginal_distribution(marg)
p1_4.conditional_distribution(cond)

print(p1)
print(p1_1)
print(p1_4)

+------+--------+
| S    |   P(S) |
+======+========+
| S(0) | 0.8842 |
+------+--------+
| S(1) | 0.1158 |
+------+--------+
+------+--------+
| S    |   P(S) |
+======+========+
| S(0) | 0.3667 |
+------+--------+
| S(1) | 0.6333 |
+------+--------+
+------+--------+
| S    |   P(S) |
+======+========+
| S(0) | 0.9538 |
+------+--------+
| S(1) | 0.0462 |
+------+--------+


In [45]:
p1_1 = JPD(['E','S','C'],[2,2,2],np.dot(invalpha_1,joint_M1_1))
p1_1.marginal_distribution(['S'])
print(p1_1)

+------+--------+
| S    |   P(S) |
+======+========+
| S(0) | 0.2000 |
+------+--------+
| S(1) | 0.8000 |
+------+--------+
